# Generate CSV file first using Geant4
Simulation for Cam3 five frames  
```
./Application_Main 10782150
```

```
Output:detect_result_256_10782150_co.csv, Time elapsed:244[s] 
```


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
DATA_FOLDER = "./data/"
CAM_NAME = "camera3"

# DATA_FOLDER = "./data2/"
# CAM_NAME = "camera2"

# DATA_FOLDER = "./data0/"
# CAM_NAME = "camera0"

DATA_FOLDER = "./data1/"
CAM_NAME = "camera1"



DATA_FOLDER = "./data_xu/"
CAM_NAME = "camera"
%matplotlib inline

## Calculate number of gamma-ray per frame

In [ ]:
m = 1.0
cm = 0.01*m
mm = 0.1*cm
um = 0.001*mm

length = 1*m
# print(f'{length / cm} cm, {length / mm} mm, {length / um} um')

# position
dist = 800*mm # camera 0
# dist = 1510*mm # camera 2
dist = 1240*mm # camera 3
dist = 1230*mm # camera 1

dist = 4000*mm # camera 1

PBq = 1.0*10**15
fps = 24
num_gamma_frame = 1 * 1 * PBq / fps #　how many gamma-ray per 1 frame？ (Co60:2, Cs137:1)

# plane is facing radiation source
# calculated by utilizing steradian
area =  2.0*mm * 2.0 * mm
# area =  0.5*mm * 0.5 * mm
sterad = area / (4*np.pi*dist*dist)
print('sterad:', sterad)
print(f'/run/beamOn {int(num_gamma_frame * sterad)}')
event_offset = int(num_gamma_frame * sterad)

In [ ]:
event_offset

In [ ]:
# /run/beamOn 1095818

## Load and parse CSV

In [ ]:
# Camera 3
fname = f"../build/detect_result_256_10782150_co_nt_Pixel_eDep.csv"


fname = f"./detect_xu_result_960x540_left_100000000_cs_nt_Pixel_eDep.csv"

In [ ]:
def parse_line(line):
    event_id, x, y, keV = line.strip().split(',')
    event_id = int(event_id)
    x = int(x)
    y = int(y)
    keV = float(keV)
    return event_id, x, y, keV

In [ ]:
# Load csv file
with open(fname) as f:
    lines = f.readlines()

noise_imgs = []
hist_keV = {}
# event_offset = 100000000
# event_offset = 5180824 # camera 0
# event_offset = 1454202 # camera 2
# event_offset = 2156430 # camera 3

img_size = (128,128)
img_size = (256,256)
img_size = (540,960)
for line in lines:
    if line[0] == '#':
        continue
    # parse line
    event_id, x, y, keV = parse_line(line)
    
    # add images
    while event_offset * len(noise_imgs) < event_id:
        noise_imgs.append(np.zeros(img_size))
    noise_imgs[-1][y, x] += keV

    if not event_id in hist_keV:
        hist_keV[event_id] = keV
    else:
        hist_keV[event_id] += keV
print(f'Total {len(noise_imgs)} frames')

In [ ]:
sum(hist_keV.values())

In [ ]:
event_id

In [ ]:
for it in noise_imgs:
    cv2.imshow("noise", it)
    key = cv2.waitKey()
    if key == 27:
        break
cv2.destroyAllWindows()

In [ ]:
plt.imshow(noise_imgs[0]>1)

In [ ]:
DATA_FOLDER

In [ ]:
# Save simulated noise images
for i, it in enumerate(noise_imgs):
    npy_fname = f'{DATA_FOLDER}geant4_gamma_noise_{i}.npy'
    np.save(npy_fname, it)
    if i%5 == 0:
        print('save noise:', npy_fname)
plt.imshow(noise_imgs[0])

In [ ]:
plt.imshow(it, vmin=0, vmax=255)

In [ ]:
##############################
## Xu paper until here
##############################

In [ ]:
# Load noise only image
gt_only_noises = []
for i in range(5):
    gt_only_noises.append(np.load(f"{DATA_FOLDER}gt_only_noise_{i}.npy"))
plt.imshow(gt_only_noises[0], vmin=0, vmax=255)

# Estimate optim parameters for converter

In [ ]:
from converter import optim_func

In [ ]:
kwargs = {}
kwargs['noise_img'] = noise_imgs[0]
kwargs['gt_noise'] = gt_only_noises[0]
kwargs["max_val"] = 4 # in keV
kwargs["min_val"] = 0  # in keV
kwargs["blur_sigma"] = 0.5
kwargs["jpeg_quality"] = 24
score, geant4_noise, noise_gt = optim_func(return_img=True, **kwargs)
fig, ax = plt.subplots(1, 2, figsize=(8,8))
ax[0].imshow(geant4_noise, vmin=0, vmax=255)
ax[1].imshow(noise_gt, vmin=0, vmax=255)
print(score)


## Grid search

22491/22491 [03:36<00:00, 103.93it/s]

In [ ]:
from sklearn.model_selection import ParameterGrid
param_grid = {
#     'max_val': np.linspace(1, 10, num=91), 
    'max_val': np.linspace(1, 6, num=51), 
    'blur_sigma': np.linspace(0.0, 1.0, num=21),
    'jpeg_quality':list(map(int, np.linspace(0, 100, num=21)))
}

In [ ]:
from tqdm.notebook import tqdm
best_score = np.inf
best_param = None
kwargs = {}
kwargs['noise_img'] = noise_imgs[0]
kwargs['gt_noise'] = gt_only_noises[0]
kwargs["min_val"] = 0

for param in tqdm(ParameterGrid(param_grid)):
    kwargs.update(param)
    score = 0
    for i in range(5):
        kwargs['gt_noise'] = gt_only_noises[i]
        tmp_score = optim_func(**kwargs)
    score = score + tmp_score
    if best_score > score:
        print(f'update score:{best_score} -> {score}')
        best_score = score
        best_param = dict(param)
        
print(best_score)
print(best_param)

In [ ]:
print(best_param)
kwargs = {}
kwargs['noise_img'] = noise_imgs[0]
kwargs['gt_noise'] = gt_only_noises[0]
kwargs.update(best_param)

score, geant4_noise, noise_gt = optim_func(return_img=True, **kwargs)
fig, ax = plt.subplots(1, 2, figsize=(8,8))
ax[0].imshow(geant4_noise, vmin=0, vmax=255)
ax[0].set_title("geant4 noise")
ax[1].imshow(noise_gt, vmin=0, vmax=255)
ax[1].set_title("real noise")
print(score)

# Save optim parameters

In [ ]:
import json
with open(f'convert_params_{cam_name}.json', 'w') as f:
    json.dump(best_param, f)

# Comparison
## Salt-and-pepper noise

In [ ]:
from scipy.stats import binom
# Salt-and-pepper noise
img_size = gt_only_noises[0].shape
p = 0.01
salt_pepper = binom.rvs(n=1, p=p, loc=0, size=img_size)
plt.imshow(salt_pepper)

In [ ]:

def add_salt_and_pepper(wo_noise, p=0.1):
    s_p_noisy = wo_noise.copy()
    salt_pepper = binom.rvs(n=1, p=p, loc=0, size=img_size)
    s_p_noisy[salt_pepper==1] = 255
    return s_p_noisy

In [ ]:
from converter import calcEMD

def optim_func_sp(return_img=False, **kwargs):
    # gt noise
    gt_noise = kwargs["gt_noise"].copy()
    hist_gt = cv2.calcHist([gt_noise], [0], None, [32], [0, 256]).flatten()
    
    geant4_noise = np.zeros_like(gt_noise)
    geant4_noise = add_salt_and_pepper(geant4_noise, p=kwargs["prob"])

    # calc histogram and compare
    if geant4_noise.dtype != np.uint8:
        geant4_noise = (255 * geant4_noise).astype(np.uint8)
    hist_noise = cv2.calcHist([geant4_noise], [0], None, [32], [0, 256]).flatten()


    dist = calcEMD(hist_noise, hist_gt)
    #     dist = calcCORREL(hist_noise, hist_gt)

    if return_img:
        return dist, geant4_noise, gt_noise
    else:
        return dist


In [ ]:
from sklearn.model_selection import ParameterGrid
param_grid = {
    'prob': np.linspace(0, 0.1, num=101),
}

In [ ]:
from tqdm.notebook import tqdm
best_score = np.inf
best_param = None
kwargs = {}
kwargs['gt_noise'] = gt_only_noises[0]

for param in tqdm(ParameterGrid(param_grid)):
    kwargs.update(param)
    score = 0
    for i in range(5):
        kwargs['gt_noise'] = gt_only_noises[i]
        tmp_score = optim_func_sp(**kwargs)
    score = score + tmp_score
    if best_score > score:
        print(f'update score:{best_score} -> {score}')
        best_score = score
        best_param = dict(param)
        
print(best_score)
print(best_param)

In [ ]:
kwargs.update(best_param)
dist, geant4_noise, gt_noise = optim_func_sp(return_img=True, **kwargs)
fig, ax = plt.subplots(1, 2)
ax[0].imshow(geant4_noise)
ax[1].imshow(gt_noise)

In [ ]:
import json
with open(f'convert_params_{CAM_NAME}_salt_pepper.json', 'w') as f:
    json.dump(best_param, f)